# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover Map

### Generate True Model, Full POVM and Inital

In [14]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, noise=0):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    targets_spam = measurement(state, U_basis = inputs_spam, povm = spam_target.povm.povm)

    #add noise
    targets_spam = add_noise_to_probs(targets_spam, noise=noise)
    return inputs_spam, targets_spam


def generate_map_data(channel_target, spam_target, N_map=None, noise=0):
    n = channel_target.n
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None,  
                                            trace=False,
                                            N=N_map)
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    state = tf.repeat(tf.expand_dims(spam_target.init.init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = channel_target.apply_channel(state)
    targets_map = measurement(state, U_basis, spam_target.povm.povm)
    
    #add noise
    targets_map = add_noise_to_probs(targets_map, noise=noise)
    
    return inputs_map, targets_map


def model_pipeline(channel_target, noise):
    # Make Benchmark
    #################################################################################
    n = 3
    d = 2**n
    
    spam_target = generate_spam_benchmark(n=n, c1=0.8, c2=0.8)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, noise=noise)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=2000, noise=noise)
    #################################################################################

    # Fit Models
    #################################################################################
    spam_model = SPAM(init = InitialState(d, c = None),
                      povm = CorruptionMatrix(d, c = None),
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(300)

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 2000,
                     verbose = False,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = d**2,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = ProbabilityMSE(),
                                            loss_function_val = channel_fidelity_loss),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = 2000,
                N = 500,
                )
    #################################################################################

    return model

In [15]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = d**2
c1 = 0.9
c2 = 0.9

#prep error and full POVM error

spam_target = SPAM(init = InitialState(d, c=c1),
                   povm = POVM(d, c=c2))

kraus_target = KrausMap(d, d, spam = spam_target)

### Generate Synthetic Data with Noise

In [16]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm.povm)

#add noise
targets_spam = add_noise_to_probs(targets_spam, 0.01)

inputs_map, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None,  
                                        trace=False)
U_prep, U_basis = inputs_map

N_map = U_prep.shape[0]
state = tf.repeat(tf.expand_dims(spam_target.init.init, axis=0), N_map, axis=0)
state = apply_unitary(state, U_prep)
state = kraus_target.apply_channel(state)
targets_map = measurement(state, U_basis, spam_target.povm.povm)

#add noise
targets_map = add_noise_to_probs(targets_map, 0.01)

### Fit Model, Initial and Corruption Matrix

In [18]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(init = InitialState(d, c=None),
                  povm = CorruptionMatrix(d, c=None),
                  loss_function = ProbabilityMSE(),
                  optimizer = tf.optimizers.Adam(learning_rate=0.01)
                 )

spam_model.pretrain(num_iter = 300)

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 verbose = False,
                )

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

0.00012420600839812684


In [19]:
model = ModelQuantumMap(channel = KrausMap(d = d, 
                                           rank = d**2,
                                           spam = spam_model,
                                          ),
                        loss_function = KLDiv(),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = KLDiv(),
                                        loss_function_val = channel_fidelity_loss),
                       )

model.train(inputs = inputs_map,
            targets = targets_map,
            inputs_val = None,
            targets_val = [kraus_target],
            num_iter = 2000,
            N = 500,
            )

  0%|          | 0/2000 [00:00<?, ?it/s]

219.58001704288995 -0.11552923002410304
50.641513212103945 -0.4239003460850099
13.844153097362438 -0.6937284496218791
5.6687089078685915 -0.8242624264294599
3.4612000273886325 -0.8867019643291799
2.698106368402252 -0.919879789409478
2.371984148652817 -0.9394686720256351
2.2140852728627722 -0.9518886665878735
2.128254756748277 -0.9601848527863284
2.081608971533325 -0.9660191187921249
2.049334991275929 -0.97033370558496
2.028984077451816 -0.9734094092077099
2.018610764395147 -0.9757366070646618
2.0111367279627492 -0.9774377848804299
2.002172025782099 -0.9787079431396951
2.0001941951547115 -0.9796718124306095
1.9947043841498588 -0.9804799775580046
1.9986771263317078 -0.9809981461225448
1.9951862571123598 -0.981425609074552
1.9965348439180146 -0.9818424382539864
1.9943007715941419 -0.9821559956362794
